# Pandas: 
### - Calling APIs using Python

<strong> <h2> Run The Following Code </h2> </strong>

Hit SHIFT + ENTER when your cusor is inside the cell of code.<br/>

> Import the packages

In [ ]:
import pandas as pd
import requests

## Calling API Example 1

> 1. Open this url in your browser: https://data.gov.sg/dataset/list-of-government-markets-hawker-centres
> 2. Click on the "Data API" button on top rigth corner to see resource_id for this dataset
> 3. Use **requests** package to call this API and get all first 5 rows of data

In [ ]:
# Option A - Manually construct the request URL
url_full = 'https://data.gov.sg/api/action/datastore_search?resource_id=8f6bba57-19fc-4f36-8dcf-c0bda382364d&limit=5'

response = requests.get(url_full)

In [ ]:
# Option B - (Recommneded) passing a dictionary to .get() method in requests to construct the request url
url_base = 'https://data.gov.sg/api/action/datastore_search'

parameters = {
    'resource_id' : '8f6bba57-19fc-4f36-8dcf-c0bda382364d',
    'limit': '5'
}
response = requests.get(url_base, params=parameters)

In [ ]:
# Check the url that sent to the API server
response.url

In [ ]:
# Check the status of the request
response.status_code

In [ ]:
# View the json text returned by the API server
response_dict = response.json()
response_dict

> The json text above is a nested-dictionary <br>
> Picture below shows the structure in a visual form

> ![](hawker_structure.png)

In [ ]:
# Traverse the dictionary to the level where the main records are located
response_dict['result']['total']

In [ ]:
# Traverse the dictionary to the level where the main records are located
records = response_dict['result']['records']
records

In [ ]:
# Check the number of records
len(records)

In [ ]:
# Use .json_normalize from Pandas to convert the dictionary into a DataFrame
pd.json_normalize(records)

In [ ]:
# Use .json_normalize from Pandas to convert the dictionary into a DataFrame 
# Store into a variable called "df_hawker"
df_hawker = pd.json_normalize(records)

## Calling API Example 2: with Parameters

> 1. Open this url in your browser: https://www.onemap.gov.sg/docs/#onemap-rest-apis
> 2. Understand the parameters (required and optional) of the "search" endpoint from OneMap API
> 3. Use **requests** package to call this API and get all first 5 rows of data

> ![apiexp](onemap_search.png)

In [ ]:
# Try to call the API (Case-sensitive Parameters not match)

url_base = 'https://developers.onemap.sg/commonapi/search'

parameters = {
    'searchval' : 'Adam Road Food Centre',
    'returngeom': 'Y',
    'getAddrDetails': 'Y'
}
response = requests.get(url_base, params=parameters)
response.status_code

In [ ]:
# Try to call the API (Required Parameters not provided)

url_base = 'https://developers.onemap.sg/commonapi/search'

parameters = {
    'searchval' : 'Adam Road Food Centre',
}
response = requests.get(url_base, params=parameters)
response.status_code

In [ ]:
# Try to call the API (Correctly)

url_base = 'https://developers.onemap.sg/commonapi/search'

parameters = {
    'searchVal' : 'Adam Road Food Centre',
    'returnGeom': 'Y',
    'getAddrDetails': 'Y'
}
response = requests.get(url_base, params=parameters)
response.status_code

In [ ]:
# View the json text returned by the server
response.json()

In [ ]:
# Use .json_normalize from Pandas to convert the dictionary into a DataFrame 
pd.json_normalize(response.json()['results'])

> 1. Loop through the **df_hawker** and find the geoinfo of each hawker from OneMap's SEARCH endpoint
> 2. Store all the geoinfo as a new DataFrame

In [ ]:
df_hawker_geoinfo = []

for row_index, row in df_hawker.iterrows():
    # Create a dictionary of the parameters
    parameters = {
        'searchVal' : row['name_of_centre'],
        'returnGeom': 'Y',
        'getAddrDetails': 'Y'
    }

    # Send the request via .get() method
    response = requests.get(url_base, params=parameters)

    # Check if the request is successful
    if response.status_code == 200:
        response_dict = response.json()
        # Check if there is at least 1 record
        if len(response_dict['results']) > 0:
            df_temp = pd.json_normalize(response_dict['results'])
            df_hawker_geoinfo.append(df_temp)
    else:
        print('The request failed and returned status code:' + str(response.status_code))

In [ ]:
# Concatenate the records stored in multiple DataFrames into a single DataFrame
df_hawker_geoinfo = pd.concat(df_hawker_geoinfo, axis=0, ignore_index=True)
df_hawker_geoinfo

# !! Your Turn !!

## Replace the code marked as <..> COMPLETELY with your own.
- The placeholder <..> is meant to be guidance for your answer.
- It should not be restricting your solutions for the questions.
- Feel free to add more lines or use less lines,
- One placeholder does not mean that you can only insert one line of codes.

# Question 1
> - Continue from the **df_hawker** and **df_hawker_geoinfo**, produce a new DataFrame **df_hawker_center**, <br>
> where each row is a unique hawker center with all the columns both **df_hawker** and **df_hawker_geoinfo**
> - The four columns "X", "Y", "LONGTITUDE", and "_id" must be excluded in the **df_hawker_center**
> - Change all the column names into lower case

In [ ]:
<..>

# Question 2


> Part A) <br>
> Store the first 200 records from the Excel file located at **data > 0_raw_data > hdb_postal_to_stations.xlsx** into a DataFrame, called **df_mrt_subset**

In [ ]:
df_mrt_subset = <..>

> Part B) <br>
> - Retrieve the geoinfo using OneMap's API Endpoint 'https://developers.onemap.sg/commonapi/search', <br>
> using the postal codes of the MRT stations
> - Store the geoinfo for the MRT stations' postal code into DataFrame **df_mrt_geoinfo** <br>
> 💡Hint: You will need to the remove the decimal point at the end of each postal code

In [ ]:
<..>

<..>

<..>

# Question 3

> Part A) <br>
> - Download all the records for HDB carpark info from the API https://data.gov.sg/dataset/hdb-carpark-information
> - Store the records into a DataFrame, called **df_carpark**

In [ ]:
<..>

> Part B)** - Challenging Question [Optional] <br>
> - Check the number of records in **df_carpark**
> - You will realize the total records on data.gov.sg is 2,183 records, but **df_carpark**only has 100 records
> - This is because it's common for API return a "page" of results per call. In this case, the API returns 100 records per page.
> - If you check *response.json()['result']['_links']*, you will see there are two links, the "next" link shows how to retrieve the next 100 records
> - Find a way to download all 2,183 records and store into the dataframe **df_carparks**

> hint: you can use anything that you have learnt so far to achieve this

In [ ]:
response.json()['result']['_links']

In [ ]:
<..>

In [ ]:
len(df_carparks)